In [1]:
!pip install -U huggingface_hub transformers langgraph langchain-core langgraph_codeact langchain_experimental langchain-openai langchain_huggingface


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.2/148.2 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/21

In [2]:
!pip install -U langchain-experimental

In [3]:
!pip install -U torch torchvision torchaudio


In [ ]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
import json
from huggingface_hub import InferenceClient

from google.colab import userdata
from huggingface_hub import login
token = ""

if token:
    login(token=token, add_to_git_credential=False)
else:
    print("HF_TOKEN secret not found. Public models will still be searchable, but gated/private ones may fail.")


In [5]:
# ----------------------
# Load catalogs
# ----------------------
with open("/content/catalog_utils.json", "r") as f:
    tools_catalog = json.load(f)
with open("/content/catalog_agents2.json", "r") as f:
    agents_catalog = json.load(f)

## Utility Functions

In [8]:
from typing import TypedDict, List, Dict, Any, Optional # Import Optional
import re
import json
import logging
import os
# ----------------------
# LLM client
# ----------------------
client = InferenceClient(provider="novita", api_key=token)

# ----------------------
# UNIFIED LLM CALLER
# ----------------------
def call_llm(prompt, user_content, few_shot_file=None, model="openai/gpt-oss-120b", temperature=0):
    """Wrapper to call LLM with optional few-shot examples stored in JSON."""
    few_shots = []
    if few_shot_file:
        with open(few_shot_file, "r") as f:
            few_shots = json.load(f)

    messages = [{"role": "system", "content": prompt}] + few_shots + [{"role": "user", "content": user_content}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        stream=False
    )
    return response.choices[0].message["content"]

# ----------------------
# LLM helper for JSON rewrite (clarification)
# ----------------------
def call_llm_json_rewrite(plan_json: Dict[str, Any], human_feedback: str,
                          few_shot_file: str = None,
                          model: str = "openai/gpt-oss-120b",
                          temperature: float = 0.0) -> Dict[str, Any]:
    """
    Ask the LLM to rewrite the given plan JSON based on human feedback.
    This function forces the model (via a system prompt) to return JSON only.
    Returns parsed JSON (dict) or raises ValueError if parsing fails.
    """
    messages = [{"role": "system", "content": CLARIFICATION_REWRITE_PROMPT}]
    if few_shot_file:
        try:
            with open(few_shot_file, "r", encoding="utf-8") as f:
                few_shots = json.load(f)
            messages += few_shots
        except Exception:
            pass

    messages.append({"role": "user", "content": f"Original plan JSON:\n{json.dumps(plan_json, indent=2)}"})
    messages.append({"role": "user", "content": f"Human feedback (natural language):\n{human_feedback}\n\nRewrite the plan JSON accordingly. Output JSON only."})

    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        stream=False
    )
    text = resp.choices[0].message["content"]

    # Parse JSON: best-effort
    try:
        parsed = json.loads(text)
        return parsed
    except Exception as e:
        # If parsing failed, raise so caller can handle (e.g., prompt human to rephrase)
        raise ValueError(f"LLM did not return valid JSON. Error: {e}\nRaw output:\n{text}")

# Custom LLM wrapper class
from langchain.llms.base import LLM
class CustomLLM(LLM):
    model: str = "openai/gpt-oss-120b"
    temperature: float = 0
    few_shot_file: Optional[str] = None

    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        # For agent use, we typically don't need a separate system prompt
        # as the agent framework handles that
        return call_llm("", prompt, self.few_shot_file, self.model, self.temperature)

# Initialize the custom LLM
llm = CustomLLM(
    model="openai/gpt-oss-120b",
    temperature=0,
    few_shot_file=None  # Set this if you have few-shot examples
)



In [34]:
import pandas as pd
def get_csv_info(file_path: str) -> str:
    """Get comprehensive information about the CSV file."""
    try:
        df = pd.read_csv(file_path)
        info = {
            "csv_path": file_path,
            "csv_name": os.path.basename(file_path),
            "shape": df.shape,
            "columns": list(df.columns),
            "dtypes": df.dtypes.to_dict(),
            "null_counts": df.isnull().sum().to_dict(),
            "memory_usage": df.memory_usage(deep=True).sum()
        }
        return str(info)
    except Exception as e:
        return f"Error getting CSV info: {str(e)}"


In [10]:
def update_existing_csv(file_path: str) -> str:
  """update the csv with response from LLM calls"""



In [11]:
def parse_prompt(prompt: str) -> Dict[str, Optional[str]]:
    result = {
        "audio_dir": None,
        "ground_truth_csv": None,
        "lang_code": None
    }
    path_pattern = r"['\"]?(/[^'\"]+/[^'\"]+/)['\"]?"
    csv_pattern = r"['\"]?(/[^'\"]+\.csv)['\"]?"
    lang_pattern = r"\b(bn|gu|hi|kn|ml|mr|pa|sd|si|ta|te|ur)\b"
    audio_dir_match = re.search(path_pattern, prompt)
    if audio_dir_match:
        result["audio_dir"] = audio_dir_match.group(1)
    csv_match = re.search(csv_pattern, prompt)
    if csv_match:
        result["ground_truth_csv"] = csv_match.group(1)
    lang_match = re.search(lang_pattern, prompt, re.IGNORECASE)
    if lang_match:
        result["lang_code"] = lang_match.group(1).lower()
    if not result["audio_dir"] and not result["ground_truth_csv"]:
        llm_prompt = f"""Extract the following from the prompt:
        1. Audio directory path (e.g., /path/to/audio/)
        2. Ground truth CSV path (e.g., /path/to/file.csv)
        3. Language code (e.g., hi, te)
        If any are unclear, return None for that field.
        Return a JSON object with keys 'audio_dir', 'ground_truth_csv', 'lang_code'.
        Prompt:
        """
        try:
            response = call_llm(llm_prompt, prompt)    #llm.invoke(llm_prompt)
            content = response.strip() # Use response from call_llm
            if content.startswith('```json'):                                                                       #markdown format correction
                content = content.replace('```json', '').replace('```', '').strip()
            llm_result = json.loads(content)
            for key in result:
                if not result[key]:
                    result[key] = llm_result.get(key)
        except Exception as e:
            logging.error(f"LLM prompt parsing failed: {e}")                                                         # LLM failsafe
            if audio_dir_match:
                result["audio_dir"] = audio_dir_match.group(1)
            elif "Audio_dir" in prompt:
                audio_dir_match = re.search(r"Audio_dir\s*=\s*['\"]?(/[^'\"]+/)['\"]?", prompt)
                if audio_dir_match:
                    result["audio_dir"] = audio_dir_match.group(1)

    if result["audio_dir"] and not os.path.isdir(result["audio_dir"]):
        logging.error(f"Invalid audio directory: {result['audio_dir']}")
        result["audio_dir"] = None
    if result["ground_truth_csv"] and not os.path.isfile(result["ground_truth_csv"]):
        logging.error(f"Invalid ground truth CSV: {result['ground_truth_csv']}")
        result["ground_truth_csv"] = None
    return result

In [12]:
import os
import logging
import torch
import librosa
import pandas as pd
from pydub import AudioSegment
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, AutoTokenizer

def transcribe_audio(audio_path, source_lang):
    model_ids = {
        "Hindi": "ai4bharat/indicwav2vec-hindi",
        "Tamil": "Harveenchadha/vakyansh-wav2vec2-tamil-tam-250",
        "Sanskrit": "addy88/wav2vec2-sanskrit-stt",
        "Marathi": "ai4bharat/indicwav2vec-marathi",
        "Telugu": "ai4bharat/indicwav2vec-telugu",
    }
    if source_lang not in model_ids:
        raise ValueError(f"Unsupported language for ai4bharat/indicwav2vec2: {source_lang}")
    model_id = model_ids[source_lang]
    processor = Wav2Vec2Processor.from_pretrained(model_id)
    model = Wav2Vec2ForCTC.from_pretrained(model_id)
    print(f"Transcribing {audio_path} using {model_id}")
    audio_array, sr = librosa.load(audio_path, sr=16000)
    inputs = processor(audio_array, sampling_rate=sr, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
        predicted_ids = torch.argmax(logits, dim=-1)
        emission = torch.nn.functional.log_softmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
    transcript_words = transcription[0].strip().split()
    print("TRANSCRIPT:", transcript_words)
    return " ".join(transcript_words)


def transcribe_folder_to_csv(folder_path: str, source_language: str):
    output_path = os.path.join(folder_path, "indicconf_hypothesis.csv")
    if os.path.exists(output_path):
        logging.info(f"Skipping transcription: File already exists at {output_path}")
        return output_path

    results = []
    for filename in os.listdir(folder_path):
        audio_path = os.path.join(folder_path, filename)
        file_size = os.path.getsize(audio_path)
        if file_size == 0:
            logging.warning(f"Skipping {filename}: File size is 0 bytes")
            continue

        delete_after = False

        if filename.lower().endswith(".mp3"):
            try:
                wav_filename = os.path.splitext(filename)[0] + ".wav"
                wav_path = os.path.join(folder_path, wav_filename)
                audio = AudioSegment.from_mp3(audio_path)
                audio.export(wav_path, format="wav")
                logging.info(f"Converted {filename} to {wav_filename}")
                audio_path = wav_path
                transcribe_filename = filename
                delete_after = True
            except Exception as e:
                logging.error(f"Failed to convert {filename} to .wav: {e}")
                continue

        elif not filename.lower().endswith(".wav"):
            logging.warning(f"Skipping {filename}: Not a .wav or .mp3 file")
            continue
        else:
            transcribe_filename = filename

        try:
            hypothesis = transcribe_audio(audio_path, source_language)
            results.append({
                "Filename": transcribe_filename,
                "Indiconformer_Hypothesis": hypothesis
            })
            logging.info(f"Transcribed {transcribe_filename}: {hypothesis[:50]}...")
        except Exception as e:
            logging.error(f"Failed to transcribe {transcribe_filename}: {e}")
        finally:
            if delete_after:
                try:
                    os.remove(audio_path)
                    logging.info(f"Deleted temporary file: {audio_path}")
                except Exception as e:
                    logging.error(f"Failed to delete temporary .wav file {audio_path}: {e}")

    if not results:
        logging.error("No valid transcriptions generated")
        return "Error: No valid transcriptions"

    df = pd.DataFrame(results)
    df.to_csv(output_path, index=False)
    logging.info(f"Transcriptions saved to {output_path}")
    return output_path

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


## Prompts

In [26]:
TASK_DECOMPOSITION_PROMPT = """
You are a task decomposition system that maps a user's request about an audio dataset to:
1. The list of EXISTING_TASKS (from a fixed predefined agent library, outputting function names)
2. Any NEW_TASKS not covered by the existing library

Your job:
- Read the user’s request
- Select **all** predefined agents whose meaning applies, regardless of exact wording
- Add any new tasks that are not already covered by the predefined list

## OUTPUT FORMAT
Respond ONLY with a single JSON object that matches this schema exactly:
{
  "EXISTING_TASKS": ["agent_function_name", "agent_function_name", ...],
  "NEW_TASKS": [
    {
      "name": "string",
      "description": "string"
    },
    ...
  ]
}
No extra text, comments, or explanations — only valid JSON.

## SEMANTIC DEFINITIONS FOR EXISTING AGENTS
Always select an agent if the request matches the meaning below, even if different words are used.

"transcription_func": Transcription — Creating a written transcript of spoken content in the audio.
"num_speaker_func": Speaker identification/diarization — Determining who is speaking, separating speech by speaker, counting speakers, or labeling speaker turns.
"transcript_quality_agent": Transcript quality assessment — Evaluating transcription accuracy, completeness, and formatting.
"character_agent": Character/graphene calculation — Counting characters or graphemes in the transcript.
"vocab_agent": Vocabulary calculation — Counting and listing unique words in the transcript.
"language_verification_agent": Language verification — Checking if the transcript matches the expected language.
"audio_length_agent": Audio length calculation — Determining the total duration of audio files.
"silence_vad_func": Silence detection — Identifying periods of no speech in the audio using VAD.
"english_word_count_agent": English word counter — Counting English words in the transcript.
"ctc_score_agent": CTC score calculation — Computing CTC-based alignment score between audio and transcript.
"upsampling_agent": Upsampling check — Detecting artificially upsampled audio.
"valid_speaker_agent": Speaker validity — Checking if speakers are new or previously known.
"domain_checker_agent": Domain detection — Determining the speech dataset’s domain (e.g., medical, legal).
"audio_transcript_matching_agent": Forced alignment — Mapping transcript text to corresponding audio segments.
"language_identification_indiclid_agent": Language identification — Detecting the language(s) spoken in the audio.
"normalization_remove_tags_agent": Transcript normalization — Removing HTML/XML tags and formatting from transcriptions.
"llm_score_agent": Coherence/fluency scoring — Using LLM-as-a-Judge to score transcript fluency (1–10 scale).
"transliteration_agent": Transliteration — Converting Roman script words to native script.
"corruption_agent": Audio corruption detection — Detecting distortion, clipping, or other signal issues.
"extension_agent": Audio format verification — Checking that files have correct extensions (e.g., .wav).
"sample_rate_agent": Sample rate verification — Ensuring audio sample rates match requirements.
"speaker_duration_agent": Per-speaker duration measurement — Calculating total speaking time for each speaker.
"utterance_duplicate_checker_agent": Utterance duplicate check — Detecting duplicate transcriptions.
"wer_computation_agent": WER computation — Calculating Word Error Rate between reference and hypothesis transcripts.

## RULES
- Always match based on meaning, not exact wording.
- If a request includes multiple actions, include **all applicable agents**.
- Do not reword existing agent names; only output their function names in EXISTING_TASKS.
- Any requirement not covered above must be added to NEW_TASKS.
- NEW_TASKS must have a concise `name` and a clear `description`.
"""

IMPL_PLANNER_SYSTEM_PROMPT = """
You are a Tool Planning Agent for LangGraph pipeline development with CodeAct Integration.

# Your Task:
Plan how to implement only New tasks as executable code that will be run via CodeAct/Python REPL.

# Chain of Thought Process:

## Step 1: Problem Understanding
For each task, analyze:
- Is this a data processing task or an AI analysis task?
- What specific operation needs to be performed?
- What type of inputs are needed (describe generically, e.g., "CSV file path", "audio directory path", "list of transcripts").
- What outputs should be generated (describe generically, e.g., "processed CSV saved to working directory", "classification results in text form").
** Do NOT reference PipelineState or any field names here. Keep everything generic.**


## Step 2: Catalog Analysis
Check existing tools/agents:
- Can any existing tool directly fulfill this task?
- Can an existing tool be modified with minor changes?
- What functionality gaps exist?

## Step 3: Implementation Planning
For new tools/agents, determine:
1. Define the expected **generic** input types (e.g., "CSV file path", "audio directory", "configuration parameters").
2. Specify the processing approach (prefer LLM-based analysis over pure Python where possible).
3. Describe the outputs and their format in **generic terms** (e.g., "processed CSV saved to working directory", "analysis report as text file").
4. Assess technical feasibility using LLM capabilities and available libraries.
5. Outline error handling requirements.
6. If a file is produced, specify it will be saved in the working directory with a descriptive filename.
7. Do not assume or reference any specific input structure/content.
8. ** Do not mention “PipelineState,” state fields, or dependencies on prior/next steps in reasoning.**


## Step 4: Achievability Assessment
Classify new tasks as:
- **Achievable**: Can be implemented with standard LLM calls
- **Not Achievable**: Requires specialized models, hardware, or unavailable resources

# Output Format Rules:
- In **REASONING**:
 - Inputs/outputs must always be described **generically** (no mention of PipelineState).
 - Example: "Takes a CSV file path as input" / "Saves results as a new CSV **in the working directory**".

# Follow this JSON output example :
```json
{
    "REASONING": [
    {
      "task": "transcription_func",
      "problem_analysis": "The task requires generating transcriptions for all audio files in a given directory and storing the results in a CSV file. The existing agent already validates the audio directory, calls the batch transcription utility, and records the path or status of the generated CSV in the pipeline state.",
      "catalog_review": "The agent \"transcription_func\" is present in the agents catalog and directly uses the \"transcribe_folder_to_csv\" tool, which fulfills the required functionality without any modification.",
      "implementation_approach": "No new implementation is needed. The existing agent will be invoked as\u2011is. It takes the audio directory path as input, runs the batch transcription utility, and updates the pipeline state with the location or status of the transcription CSV.",
      "decision_rationale": "Since a fully functional agent already exists that matches the required behavior, we simply reuse it."
    },
    {
      "task": "Sentiment classification",
      "problem_analysis": "We need to assign a sentiment label (e.g., positive, neutral, negative) to each transcription in the dataset. The input is a CSV file containing transcriptions, and the output should be a new CSV (or an updated version) that adds a column with the sentiment label for every row.",
      "catalog_review": "No tool or agent in the current catalog performs sentiment analysis on text. The closest utilities are generic transcription and language identification tools, which do not address sentiment.",
      "implementation_approach": "Create a new agent that reads the transcription CSV, iterates over each transcript, and uses an LLM (via the generic \"llm\" tool) to classify sentiment. The agent writes the results to a new CSV file (e.g., \"sentiment_classification.csv\") and records the file path in the pipeline state.",
      "decision_rationale": "LLM\u2011based classification is achievable with the existing \"llm\" tool and avoids the need for additional heavy sentiment\u2011analysis libraries. The approach is lightweight, requires only Python for CSV handling, and fits the llm_hybrid implementation type."
    }
   ]
}

"""
TOOL_PLANNER_SYSTEM_PROMPT = """
# You are a Tool Planning Agent.

# # You are given:
# 1. A catalog of existing agents and tools functions with their descriptions, inputs, outputs, and dependencies.
# 2. A list of tasks to perform.
# 3. The PipelineState schema that defines available state variables in the LangGraph pipeline.

## Achievability Assessment
Classify new tasks as:
- **Achievable**: Can be implemented with standard LLM calls
- **Not Achievable**: Requires specialized models, hardware, or unavailable resources

# PipelineState Fields Available:
- audio_dir: str → Audio files directory
- ground_truth_csv: str → Ground truth CSV file
- transc_csv: str → Generated transcript CSV
- test_transc_csv: str → Test transcript CSV for validation
- test_audio_dir: str → Test audio directory for validation
- lang_code: str → Language code for processing
- user_request: str → Original user request

# Decision Framework:
- **Use Existing**: Tool exists and works as-is
- **Modify Existing**: Tool exists but needs input/output adjustments
- **Create New**: No suitable tool exists, but implementation is feasible

# Output Format Rules:
- In **PLAN**:
 - Inputs and outputs must explicitly reference **PipelineState fields**.
 - Always use the format:
   - "input_spec": "PipelineState - uses fields: [field1, field2]"
 - Python depecdencies are not considered as tool dependencies , they are different
 - llm is not a tool

# Final Output Format:
```json
{
  "PLAN": [
    {
      "task": "task_description",
      "use_existing": ["tool_name"],
      "modify_existing": [
        {
          "tool": "tool_name",
          "modification_description": "Specific changes needed"
        }
      ],
      "create_new": [
        {
          "name": "new_agent_name",
          "description": "What this agent does and how it works",
          "achievable": "Yes/No with technical justification",
          "input_spec": "PipelineState - uses fields: [field1, field2]",
          "output_spec": "PipelineState - updates fields: [field3, field4]",
          "implementation_type": "llm_analysis/python_only/llm_hybrid",
          "tool_dependencies": ["Tool1", "Tool2"],
          "agent_dependencies": ["existing_agent"]
        }
      ]
    }
  ]
}

## IMPLEMENTATION TYPES AND RULES:
1. llm_analysis: Primarily LLM-based with minimal Python support (Default and preferred)
2. llm_hybrid: Combines Python data processing with LLM analysis
3. python_only: Uses only Python libraries (pandas, os, etc.)
"""


# TOOL_PLANNER_SYSTEM_PROMPT = """
# You are a Tool Planning Agent.

# # You are given:
# 1. A catalog of existing agents and tools functions with their descriptions, inputs, outputs, and dependencies.
# 2. A list of tasks to perform.
# 3. The PipelineState schema that defines available state variables in the LangGraph pipeline.

# # PipelineState fields:
# - audio_dir: str → Path to folder containing audio files
# - ground_truth_csv: str → Path to ground-truth CSV file if provided
# - transc_csv: str → Path to transcript CSV file created by transcript_func
# - test_transc_csv: str → Path to test transcript CSV file for validating new tools/agents
# - test_audio_dir: str → Path to test audio directory for validating new tools/agents
# - lang_code: str → Language code for processing
# - user_request: str → Raw natural language request from the user
# - task_decomposition: dict → Structured breakdown of tasks
# - task_list_for_planner: list[str] → Flattened list of tasks passed to planner
# - tool_plan: dict → JSON representation of tool/agent plan
# - clarification_round: int → Current clarification round
# - clarification_history: list[dict] → Log of clarification interactions
# - clarification_done: bool → Whether clarification is finished
# - human_feedback: str → Freeform feedback text from user

# # For each task, decide:
# - If an existing tool/agent can be used as-is
# - If an existing tool/agent can be modified to achieve it
# - If a new tool/agent must be created

# # Output JSON only in this format:
# {
#   "PLAN": [
#     {
#       "task": "string",
#       "use_existing": ["ToolName1", "ToolName2"],
#       "modify_existing": [
#         {
#           "tool": "ToolName",
#           "modification_description": "string"
#         }
#       ],
#       "create_new": [
#         {
#           "name": "string",
#           "description": "string",
#           "input_spec": "string - must reference a PipelineState field when possible",
#           "output_spec": "string - must reference a PipelineState field when possible",
#           "tool_dependencies": ["list","of","strings","of","tool","names","from","catalog"],
#           "agent_dependencies": ["list","of","strings","of","agent","names","from","catalog"]
#         }
#       ]
#     }
#   ]
# }

# # Rules:
# - Always prefer PipelineState fields for input_spec and output_spec (e.g., `"string - Path to folder containing audio files (audio_dir) from PipelineState"`).
# - If no relevant PipelineState field exists, allow a general spec (e.g., "string - User query").
# - Only reference tools and agents from the provided catalog unless creating new.
# - Match dependencies based on functionality, not just wording similarity.
# - input_spec and output_spec :
#     - `"type - meaning + PipelineState + field"` when applicable. important to mention state name
#     - `"type - meaning "`
# - tool_dependencies: list only function names, modules, llm or libraries directly needed for the implementation.
# - agent_dependencies: list existing agents that might be needed before/after implementing this tool/agent.
# - If a task involves evaluation, scoring, grading, or subjective judgment (e.g., relevance, sentiment, fluency, coherence), check for a scoring tool in the catalog first.
#   - If none exists, create a new LLM-based scoring agent that uses an existing LLM utility function as the primary dependency.
#   - Always specify in the description that scoring is performed by an LLM with a well-defined prompt.

# """


In [27]:
CLARIFICATION_REWRITE_PROMPT = """
You are a JSON rewrite assistant.

You will be given:
1) Original tool plan JSON.
2) Natural language human feedback.

Task:
- Incorporate the human feedback into the original plan by modifying only the necessary parts.
- Keep the exact JSON structure:
```json
{
  "REASONING": [
    {
      "task": "task_name",
      "problem_analysis": "What exactly needs to be done?",
      "catalog_review": "What existing tools were considered?",
      "implementation_approach": "How will this be implemented?",
      "decision_rationale": "Why this approach was chosen?"
    }
  ],
  "PLAN": [
    {
      "task": "task_description",
      "use_existing": ["tool_name"],
      "modify_existing": [
        {
          "tool": "tool_name",
          "modification_description": "Specific changes needed"
        }
      ],
      "create_new": [
        {
          "name": "new_agent_name",
          "description": "What this agent does and how it works",
          "achievable": "Yes/No with technical justification",
          "input_spec": "PipelineState fields: [field1, field2]",
          "output_spec": "PipelineState fields: [field3, field4]",
          "implementation_type": "python_only/llm_hybrid/llm_analysis",
          "tool_dependencies": ["library1", "library2"],
          "agent_dependencies": ["existing_agent"]
        }
      ]
    }
  ]
}
```
Field Descriptions:
- **task**: A short description of the specific task to be performed.
- **use_existing**: List of existing tools that will be directly used without modification.
- **modify_existing**: List of modifications to existing tools (tool name + description of change).
- **create_new**: List of entirely new tools that need to be created. Each new tool requires:
  - **name**: Unique name of the new tool.
  - **description**: Purpose and functionality of the tool.
  - **input_spec**: Description of input format and requirements.
  - **output_spec**: Description of output format and expectations.
  - **tool_dependencies**: List of other tools this tool depends on.
  - **agent_dependencies**: List of agent processes or agents this tool depends on.

**Important:**
- Output **only valid JSON** (no extra text, no commentary).
- Preserve unchanged entries unless the human feedback requires a change.
- If feedback asks to merge or remove items, reflect that exactly in the JSON.
- The three fields **use_existing, modify_existing, create_new** are exhaustive.

"""

## Python Tool

In [14]:
from langchain.tools import Tool
from langchain_experimental.utilities import PythonREPL
from langchain.agents import initialize_agent, AgentType

python_repl = PythonREPL()
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)
tools = [repl_tool, ]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    verbose=True,
    handle_parsing_errors=True,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

/tmp/ipython-input-406648470.py:12: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


## State

In [15]:
from typing import TypedDict, List, Dict, Any
# ----------------------
# State type
# ----------------------
class PipelineState(TypedDict, total=False):
    audio_dir: str
    ground_truth_csv: str
    transc_csv: str
    lang_code: str
    user_request: str
    task_decomposition: Dict[str, Any]
    task_list_for_planner: List[str]
    impl_plan: Dict[str, Any]
    tool_plan: Dict[str, Any]
    clarification_round: int
    clarification_history: List[Dict[str, Any]]
    clarification_done: bool
    validation_done: bool
    human_feedback: str  # Add this field

## LangGraph Agent

In [20]:
# ----------------------
# LANGGRAPH NODES
# ----------------------

def task_decomposition_node(state):
    """Node 1: Decompose user request into tasks."""
    print("---Task Decomposition---")
    user_request = state["user_request"]
    print(f"--------------------Calling LLM--------------------")
    task_decomp_output = call_llm(
        TASK_DECOMPOSITION_PROMPT,
        user_request,
        few_shot_file="/content/task_decomposer.json"
    )
    print(f"--------------------LLM output--------------------")
    print(task_decomp_output)

    tasks_data = json.loads(task_decomp_output)
    task_list_for_planner = list(tasks_data["EXISTING_TASKS"]) + [t["name"] for t in tasks_data["NEW_TASKS"]]

    return {
        "task_decomposition": tasks_data,
        "task_list_for_planner": task_list_for_planner
    }

def impl_planning_node(state):
    """Node 2: Plan tools/agents implementation based on tasks."""
    print("---Tool Planning---")
    impl_planner_input = json.dumps({
        "tools_catalog": tools_catalog,
        "agents_catalog": agents_catalog,
        "tasks": state["task_list_for_planner"]
    }, indent=2)

    print(f"--------------------Calling LLM--------------------")
    impl_plan_output = call_llm(
        IMPL_PLANNER_SYSTEM_PROMPT,
        impl_planner_input,
        few_shot_file=None
    )
    raw = impl_plan_output.strip()
    print(f"Implementation plan: {raw}")

    try:
        impl_plan = json.loads(raw)
    except json.JSONDecodeError:
        match = re.search(r"\{.*\}", raw, re.DOTALL)
        if match:
            try:
                impl_plan = json.loads(match.group(0))
            except json.JSONDecodeError:
                raise ValueError(f"Extracted JSON block is invalid: {match.group(0)}")

    return {"impl_plan": impl_plan}

def tool_planning_node(state):
    """Node 2: Plan tools/agents based on tasks."""
    print("---Tool Planning---")
    tool_planner_input = json.dumps({
        "tools_catalog": tools_catalog,
        "agents_catalog": agents_catalog,
        "tasks": state["task_list_for_planner"]
    }, indent=2)

    print(f"--------------------Calling LLM--------------------")
    tool_plan_output = call_llm(
        TOOL_PLANNER_SYSTEM_PROMPT,
        tool_planner_input,
        few_shot_file="/content/tool_planner.json"
    )
    raw = tool_plan_output.strip()
    print(f"Tool plan: {raw}")

    try:
        tool_plan = json.loads(raw)
    except json.JSONDecodeError:
        match = re.search(r"\{.*\}", raw, re.DOTALL)
        if match:
            try:
                tool_plan = json.loads(match.group(0))
            except json.JSONDecodeError:
                raise ValueError(f"Extracted JSON block is invalid: {match.group(0)}")

    return {"tool_plan": tool_plan}

# ----------------------
# Node 3: Clarification (HITL) - CORRECTED VERSION
# ----------------------

def clarification_request_node(state):
    """
    Ask human to review tool plan and provide feedback.
    Uses interrupt() to pause execution and wait for human input.
    """
    print("---Clarification Request---")
    current_impl_plan = state.get("impl_plan", {})
    current_tool_plan = state.get("tool_plan", {})
    round_number = state.get("clarification_round", 0) + 1

    prompt_text = (
        f"=== Clarification Round {round_number} ===\n\n"
        "Please review the current implementation + tool plan below and respond with:\n"
        "- 'approve' or 'ok' to accept the plan, OR\n"
        "- Natural language instructions to change the plan.\n\n"
        "Current impl plan:\n"
        f"{json.dumps(current_impl_plan, indent=2)}\n\n"
        "Current tool plan:\n"
        f"{json.dumps(current_tool_plan, indent=2)}\n\n"
        "Your feedback:"
    )

    # Use interrupt() to pause and wait for human feedback
    feedback = interrupt(prompt_text)

    # Return the feedback to be stored in state
    return {
        "human_feedback": feedback,
        "clarification_round": round_number
    }


def clarification_process_node(state):
    """
    Process human feedback, update plan if needed.
    """
    print("---Processing Clarification---")
    current_plan = copy.deepcopy(state.get("impl_plan", {}))
    feedback_text = (state.get("human_feedback") or "").strip()
    round_number = state.get("clarification_round", 0)

    # If approved → finish
    if feedback_text.lower() in {"ok", "approve", "approved", "yes", "looks good"}:
        history_entry = {
            "round": round_number,
            "feedback": feedback_text,
            "action": "approved",
            "plan_before": current_plan,
            "plan_after": current_plan
        }
        clarification_history = state.get("clarification_history", []) + [history_entry]
        return {
            "clarification_history": clarification_history,
            "clarification_done": True
        }

    # Otherwise → rewrite plan with LLM
    rewritten_plan = call_llm_json_rewrite(
        plan_json=current_plan,
        human_feedback=feedback_text,
        few_shot_file=None,
        model="openai/gpt-oss-20b",
        temperature=0.0
    )

    history_entry = {
        "round": round_number,
        "feedback": feedback_text,
        "action": "rewritten",
        "plan_after": rewritten_plan
    }

    return {
        "tool_plan": rewritten_plan,
        "clarification_history": state.get("clarification_history", []) + [history_entry],
        "clarification_done": False
    }


In [21]:
def transcription_func(state: PipelineState):
    audio_dir = state.get('audio_dir')
    if not audio_dir or not os.path.isdir(audio_dir):
        logging.error(f"Invalid audio directory for transcription: {audio_dir}")
        return {"A": "Error: Invalid audio directory"}
    logging.info("Running Transcription")
    result = transcribe_folder_to_csv(audio_dir, source_language="Hindi")
    return {"transc_csv": result} #, "audio_dir": audio_dir

In [22]:
import re
import json
import copy
from typing import TypedDict, List, Dict, Any
from langgraph.graph import StateGraph, END
from langgraph.constants import START
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import InMemorySaver

def clarification_condition(state):
    """Decide whether to continue or end clarification loop."""
    if state.get("clarification_done", False) or state.get("clarification_round", 0) >= 3:
        return END
    return "clarification_request"


# Build state graph
graph = StateGraph(PipelineState)

# Add nodes
graph.add_node("decompose_task_node", task_decomposition_node) # Renamed node
graph.add_node("impl_planning", impl_planning_node)
graph.add_node("tool_planning", tool_planning_node)
graph.add_node("clarification_request", clarification_request_node)
graph.add_node("clarification_process", clarification_process_node)

# Add edges
graph.add_edge(START, "decompose_task_node") # Updated edge
graph.add_edge("decompose_task_node", "impl_planning") # Updated edge
graph.add_edge("impl_planning", "tool_planning") # Updated edge
graph.add_edge("tool_planning", "clarification_request")
graph.add_edge("clarification_request", "clarification_process")

# Add conditional edge for clarification loop
graph.add_conditional_edges("clarification_process", clarification_condition)

# CRITICAL: Add checkpointer for human-in-the-loop to work
memory = InMemorySaver()
app = graph.compile(checkpointer=memory)

In [23]:
def run_pipeline_with_hitl(user_request: str):
    """Run the pipeline with proper human-in-the-loop handling."""

    # Create thread configuration
    thread_config = {"configurable": {"thread_id": "pipeline_thread_1"}}

    # Initial state
    initial_state = {"user_request": user_request, "clarification_round": 0, "validation_done": False}

    print("Starting pipeline execution...")

    # Stream the graph execution
    for event in app.stream(initial_state, thread_config, stream_mode="updates"):
        print(f"Event: {event}")

        # Check if we hit an interrupt
        if "__interrupt__" in event:
            interrupt_data = event["__interrupt__"][0]
            print(f"\n HUMAN INPUT REQUIRED:")
            print(f"Prompt: {interrupt_data.value}")

            # Get human feedback
            human_feedback = input("\nYour feedback: ").strip()

            # Resume execution with the feedback
            print(f"\n Resuming with feedback: {human_feedback}")

            # Continue streaming with the human feedback
            for resume_event in app.stream(
                Command(resume=human_feedback),
                thread_config,
                stream_mode="updates"
            ):
                print(f"Resume Event: {resume_event}")

                # Handle nested interrupts (if clarification loops)
                if "__interrupt__" in resume_event:
                    nested_interrupt = resume_event["__interrupt__"][0]
                    print(f"\n ADDITIONAL INPUT REQUIRED:")
                    print(f"Prompt: {nested_interrupt.value}")

                    nested_feedback = input("\nYour feedback: ").strip()
                    print(f"\n Resuming with feedback: {nested_feedback}")

                    # Continue with nested feedback
                    for final_event in app.stream(
                        Command(resume=nested_feedback),
                        thread_config,
                        stream_mode="updates"
                    ):
                        print(f"Final Event: {final_event}")

            break  # Exit the main loop after handling interrupt

    # Get final state
    final_state = app.get_state(thread_config)
    print("\n=== Final State ===")
    print(json.dumps(final_state.values, indent=2))

    return final_state

## Run

In [29]:
user_request = ("I have audio file directory at /content/audios and i need to generate transcript of each audio, and want to do sentiment analysis on the transcripts.")#evaluate the relevance of answer to the question from transcripts ") #


In [51]:
run_pipeline_with_hitl(user_request)

Starting pipeline execution...
---Task Decomposition---
--------------------Calling LLM--------------------
--------------------LLM output--------------------
{
  "EXISTING_TASKS": ["transcription_func"],
  "NEW_TASKS": [
    {
      "name": "sentiment_analysis",
      "description": "Perform sentiment analysis on the generated transcripts."
    }
  ]
}
Event: {'decompose_task_node': {'task_decomposition': {'EXISTING_TASKS': ['transcription_func'], 'NEW_TASKS': [{'name': 'sentiment_analysis', 'description': 'Perform sentiment analysis on the generated transcripts.'}]}, 'task_list_for_planner': ['transcription_func', 'sentiment_analysis']}}
---Tool Planning---
--------------------Calling LLM--------------------
Implementation plan: ```json
{
  "REASONING": [
    {
      "task": "transcription_func",
      "problem_analysis": "The task requires converting all audio files in a given folder into text transcriptions and storing the results in a CSV file. The input is a path to an audio dire

StateSnapshot(values={'user_request': 'I have audio file directory at /content/audios and i need to generate transcript of each audio, and want to do sentiment analysis on the transcripts.', 'task_decomposition': {'EXISTING_TASKS': ['transcription_func'], 'NEW_TASKS': [{'name': 'sentiment_analysis', 'description': 'Perform sentiment analysis on the generated transcripts.'}]}, 'task_list_for_planner': ['transcription_func', 'sentiment_analysis'], 'impl_plan': {'REASONING': [{'task': 'transcription_func', 'problem_analysis': 'The task requires converting all audio files in a given folder into text transcriptions and storing the results in a CSV file. The input is a path to an audio directory and a language identifier for the ASR model. The output is a CSV file containing at least the filename and its corresponding transcription.', 'catalog_review': 'A utility function named "transcribe_folder_to_csv" already exists. It accepts an audio folder path and a source language, processes each au

In [35]:
thread_config = {"configurable": {"thread_id": "pipeline_thread_1"}}

# Get current state
state = app.get_state(thread_config)
current_values = state.values

# Access specific elements
user_request = current_values.get("user_request")
impl_plan = current_values.get("impl_plan")
tool_plan = current_values.get("tool_plan")
task_decomposition = current_values.get("task_decomposition")
clarification_round = current_values.get("clarification_round", 0)
clarification_done = current_values.get("clarification_done", False)

print(f"Current round: {clarification_round}")
print(f"Impl plan: {json.dumps(impl_plan, indent=2)}")
print(f"Tool plan: {json.dumps(tool_plan, indent=2)}")

Current round: 1
Impl plan: {
  "REASONING": [
    {
      "task": "transcription_func",
      "problem_analysis": "The task requires converting all audio files in a given folder into text transcriptions and storing the results in a CSV file. The input is a path to an audio folder and a language identifier. The output is a CSV file that contains at least the filename and its corresponding transcription.",
      "catalog_review": "The tool catalog includes a function named \"transcribe_folder_to_csv\" that batch\u2011processes a folder of audio files, performs language\u2011specific ASR, and writes the results to a CSV file. This tool directly satisfies the required functionality without modification.",
      "implementation_approach": "No new implementation is needed. The existing \"transcribe_folder_to_csv\" tool will be invoked with the audio folder path and the chosen source language. It will generate a CSV file (e.g., \"indicconf_hypothesis.csv\") in the working directory containin

In [ ]:
# workflow agent - > ask which tools to execute
#

## Generation and testing simpler

In [42]:
from langchain_openai import ChatOpenAI
from langgraph_codeact import create_codeact
import builtins
import contextlib
import io
from typing import Any

# 1. Load your LLM (using HuggingFace router as an example)
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    openai_api_base="https://router.huggingface.co/v1",
    openai_api_key=token,
    model="openai/gpt-oss-120b:novita"
)
# 2. Define or collect your tool functions (empty for generic tasks, can add more)
tools = [get_csv_info]  # Add functions as needed, e.g. [my_tool_func]

# 3. Define a code sandbox (secure in production; this is for demonstration)
import contextlib, io, builtins, logging, traceback
from typing import Any

def eval(code: str, _locals: dict[str, Any], logger=None) -> tuple[str, dict[str, Any]]:
    logger = logger or logging.getLogger(__name__)
    original_keys = set(_locals.keys())
    try:
        try:
            logger.info("Compiling code before execution to check syntax errors")
            compile(code, "<string>", "exec")
        except SyntaxError as se:
            logger.error(f"Generated code has a syntax error: {se}")
            logger.error(f"Full generated code:\n{code}")
            raise
        with contextlib.redirect_stdout(io.StringIO()) as f:
            exec(code, builtins.__dict__, _locals)
        result = f.getvalue() or "<code ran, no output printed to stdout>"
    except Exception as e:
        result = f"Error during execution: {repr(e)}\n{traceback.format_exc()}"
    new_keys = set(_locals.keys()) - original_keys
    new_vars = {key: _locals[key] for key in new_keys}
    return result, new_vars


# 4. Create the CodeAct agent
codeact_agent = create_codeact(llm, tools, eval).compile()


In [53]:
import os
import sys
import json
import logging
import traceback
from typing import Any, Dict

def gen_and_execute_dynamic_task(state: Dict[str, Any], task_name: str, codeact_agent, logger=None, max_retries=3):
    """
    Node to dynamically generate and execute code for a new task as described in the tool plan using CodeAct agent.
    Args:
        state: Pipeline state dict containing 'impl_plan' and 'tool_plan' with input fields.
        task_name: Name of the task to generate and execute.
        codeact_agent: Compiled CodeAct agent instance.
        logger: Optional logging.Logger instance.
        max_retries: Number of retries for reflection loop.
    Returns:
        Updated pipeline state.
    """
    logger = logger or logging.getLogger(__name__)
    impl_plan = state.get("impl_plan")
    tool_plan = state.get("tool_plan")
    if not tool_plan:
        logger.warning("No tool plan in state!")
        return state
    if not impl_plan:
        logger.warning("No impl plan in state!")
        return state

    # Find new task definition
    new_task = None
    for t in tool_plan.get("PLAN", []):
        if t["task"] == task_name and t.get("create_new"):
            new_task = t["create_new"][0]  # Assume only one new tool per task
            break
    if not new_task:
        logger.info(f"No new task named {task_name} to create.")
        return state

    # Find reasoning for this task
    reasoning = ""
    for reasoning_item in impl_plan.get("REASONING", []):
        if reasoning_item.get("task") == task_name:
            reasoning = json.dumps(reasoning_item, indent=2)
            break

    # Parse input/output specs (field names)
    input_spec = re.findall(r"\[(.*?)\]", new_task.get("input_spec", ""))
    # output_spec = re.findall(r"\[(.*?)\]", new_task.get("output_spec", ""))
    func_name = new_task.get("name", f"{task_name}_function")

    # Prepare input parameter values
    input_params = {key: state[key] for key in input_spec if key in state}
    if len(input_params) != len(input_spec):
        missing = set(input_spec) - set(input_params)
        logger.error(f"Missing required input parameters for task '{task_name}': {missing}")
        # state['status'] = f"fail: missing input {missing}"
        return state

    file_path = input_params.get(input_spec[0])          ############################################## CHECK with this #################
    # Build allowed tools/deps string for prompt
    allowed_deps = new_task.get("tool_dependencies", [])
    allowed_deps.remove('llm')                        #############################################33 CHECK with this ################3
    allowed_tools_str = ", ".join(allowed_deps) if allowed_deps else "standard Python + pandas, csv, os, logging"

    # Strict param signature for function
    # param_str = ", ".join([f"{p}: str" for p in input_spec])
    # output_key = output_spec[0] if output_spec else "output"

    csv_info  = get_csv_info(file_path)
    generation_prompt = f"""
    # CONTEXT
    You are an intelligent CSV Analyzer and a PYthon Developer Agent with the ability to perform comprehensive data analysis on CSV files.

    **Your Capabilities:**
    - You have access to a suite of pre-built  tools (functions) that you can call directly
    - You can write and execute custom Python code to perform any analysis not covered by existing tools
    - You can combine tool usage with custom code for complex analytical workflows
    - You excel at data exploration, pattern recognition, and insight generation

    Below is the reasoning, specs, and contract for a new task.You must generate a Python function that fulfills the requirements, using best practices.

    # REASONING
    {reasoning}

    # IMPLEMENTATION CONTRACT
    - Function name: {func_name}
    - Input csv info: {csv_info}
    - Output: save the updated csv in working directory and return/log output path as a string (raise/log on errors)
    - Working directory: You may read/write files in the current directory.
    - Error handling: Log exceptions, use safe defaults if possible.
    - The code must define a single function named `{func_name}`. Do not define any other functions or classes.
    - Only output the function definition, fully self-contained in a code block.
    - call existing tools directly inside the function definition



    # INSTRUCTIONS
    ** WHENEVER YOU GENERATE CODE , ALWAYS WRITE CODE IN BETWEEN BACTICKS : ``` **
    1. All required imports (including `os`, `sys`, `json`, `logging`, `pandas`, etc.) must be included at the top of the generated function. Do not assume any package is already imported.
    2. All code must be a complete, syntactically valid Python function that can be executed with exec().
    3. If you use exception handling, always provide both the try and except blocks, properly indented inside the function.
    4. Do not output code snippets—output the full function definition.
    5. Implement the functionality described in the reasoning and specs above.
    6. If file I/O is required, use the provided paths.
    7. If you need to call an LLM, use the `llm` tool (if provided).
    8. Place all new output files in the current working directory.
    10. After code generation, log the code for traceability.( very important )
    11. please do not use invalid character in the code . example invalid character '✅' (U+2705)

    # BEGIN CODE
    """.strip() # - Allowed dependencies/tools: {allowed_tools_str}
    # Reflection/correction loop
    last_error = None
    for attempt in range(max_retries):
        logger.info(f"[{task_name}] Code generation attempt {attempt+1}")
        # Prepare agent state for codeact
        agent_state = {
            "messages": [
                {
                    "role": "user",
                    "content": (
                        generation_prompt +
                        ("\n\n# PREVIOUS ERROR:\n" + str(last_error) if last_error else "")
                    )
                }
            ]
        }
        # Generate code using CodeAct agent
        try:
            response = codeact_agent.invoke(agent_state)
            code = None
            for msg in response.get("messages", []):
                if msg.get("role") == "assistant" and "```" in msg.get("content", ""):
                    # Extract code block
                    code_block = re.search(r"```(?:python)?\n(.*?)```", msg["content"], re.DOTALL)
                    if code_block:
                        code = code_block.group(1).strip()
                        break
            if not code:
                logger.error(f"[{task_name}] No code block generated by agent.")
                last_error = "No code block generated."
                continue

            logger.info(f"[{task_name}] Generated code:\n{code}")

            # Prepare execution environment
            local_env = {}
            # Compile code
            exec(code, {}, local_env)
            func = local_env.get(func_name)
            if not func or not callable(func):
                logger.error(f"[{task_name}] Generated code did not define {func_name}.")
                last_error = f"Generated code did not define {func_name}."
                continue

            # Call function with strict params
            logger.info(f"[{task_name}] Executing generated function: {func_name}")
            output_path = func(**input_params)
            # Check output file exists
            if output_path and os.path.exists(output_path):
                logger.info(f"[{task_name}] Output file created at: {output_path}")
                state[output_key] = output_path
                state['status'] = "success"
                return state
            else:
                logger.error(f"[{task_name}] Output file not found after execution: {output_path}")
                last_error = f"Output file not found: {output_path}"

        except Exception as e:
            tb = traceback.format_exc()
            logger.error(f"[{task_name}] Error during code generation/execution: {e}\n{tb}")
            last_error = f"{e}\n{tb}"

    # If all retries fail
    logger.error(f"[{task_name}] Failed to perform task after {max_retries} attempts. Last error: {last_error}")
    state['status'] = "fail: unable to perform task"
    state['error'] = str(last_error)
    return state



In [54]:
# 3. Example pipeline state and tool_plan (for sentiment analysis step)
state = PipelineState(
        audio_dir="sample_audio/",
        ground_truth_csv="ground_truth.csv",
        transc_csv="/content/transcript_file.csv",
        lang_code="en",
        user_request="Perform sentiment analysis on the transcriptions.",
        task_decomposition={},
        task_list_for_planner=[],
        impl_plan ={
             "REASONING": [
                    {
                      "task": "sentiment_classification",
                      "problem_analysis": "The task is to assign a sentiment label (e.g., positive, neutral, negative) to each transcription in a dataset. The input is a CSV file that contains a column with transcription text. The output should be a new CSV file (or an updated version) that adds a column with the sentiment label for every row.",
                      "catalog_review": "No tool or agent in the current catalog performs sentiment analysis on text data. The available utilities focus on audio processing, language identification, transliteration, and forced alignment, but none provide text\u2011level classification.",
                      "implementation_approach": "Create a new agent that:\n1. Accepts the path to the transcription CSV file as input.\n2. Reads the CSV using pandas, iterates over each transcription row.\n3. For each transcription, calls the generic \"llm\" tool with a prompt that asks the model to classify the sentiment (positive/neutral/negative).\n4. Collects the returned label and appends it to a new column named \"sentiment\".\n5. Writes the enriched data to a new CSV file (e.g., \"sentiment_classification.csv\") saved in the working directory.\n6. Returns the path of the generated CSV as the output.\nError handling includes catching read/write exceptions and handling unexpected LLM responses by defaulting to \"neutral\".",
                      "decision_rationale": "LLM based sentiment classification is achievable with the existing \"llm\" tool and standard Python libraries (pandas, csv). No specialized sentiment models or hardware are required, making the implementation straightforward and compatible with the current environment."
                    }
                        ]
                    }, # Added missing comma here
        tool_plan={

  "PLAN": [
    {
      "task": "sentiment_analysis",
      "use_existing": [],
      "modify_existing": [],
      "create_new": [
        {
          "name": "sentiment_analysis_agent",
          "description": "Reads a CSV of transcriptions, uses an LLM to assign a sentiment label (positive/neutral/negative) and optional confidence score to each transcript, and writes the results to a new CSV.",
          "achievable": "Yes",
          "input_spec": "PipelineState - uses fields: [transc_csv]",
          "output_spec": "PipelineState - updates fields: [sentiment_output]",
          "implementation_type": "llm_hybrid",
          "tool_dependencies": [
            "pandas",
            "llm"
          ],
          "agent_dependencies": []
        }
      ]
    }
  ]
},
        clarification_round=0,
        clarification_history=[],
        clarification_done=True,
        validation_done=False,
        human_feedback=""
    )

# 4. Test the node
if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)
    result_state = gen_and_execute_dynamic_task(
        state,
        "sentiment_analysis",
        codeact_agent,
        logger=logging.getLogger("test_codeact")
    )
    print("Final pipeline state:", result_state)

ERROR:__main__:Generated code has a syntax error: unmatched ')' (<string>, line 97)
ERROR:__main__:Full generated code:
def sentiment_analysis_agent():
    """
    Reads '/content/transcript_file.csv', performs multilingual sentiment analysis
    on the 'Indiconformer_Hypothesis' column, writes the augmented CSV to the
    current working directory, prints the file path, and returns it.

    The function first tries to use a Hugging Face multilingual sentiment model.
    If that fails (e.g., but it will still work with a simple rule‑based fallback.
    All steps are logged; any exception is logged and re‑raised.
    """
    import os
    import sys
    import json
    import logging
    import subprocess
    import importlib
    import pandas as pd
    import numpy as np

    # ------------------------------------------------------------------ #
    # Logging configuration
    # ------------------------------------------------------------------ #
    logging.basicConfig(
        level=

Final pipeline state: {'audio_dir': 'sample_audio/', 'ground_truth_csv': 'ground_truth.csv', 'transc_csv': '/content/transcript_file.csv', 'lang_code': 'en', 'user_request': 'Perform sentiment analysis on the transcriptions.', 'task_decomposition': {}, 'task_list_for_planner': [], 'impl_plan': {'REASONING': [{'task': 'sentiment_classification', 'problem_analysis': 'The task is to assign a sentiment label (e.g., positive, neutral, negative) to each transcription in a dataset. The input is a CSV file that contains a column with transcription text. The output should be a new CSV file (or an updated version) that adds a column with the sentiment label for every row.', 'catalog_review': 'No tool or agent in the current catalog performs sentiment analysis on text data. The available utilities focus on audio processing, language identification, transliteration, and forced alignment, but none provide text‑level classification.', 'implementation_approach': 'Create a new agent that:\n1. Accepts 